In [ ]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 
from keras import layers 
import tensorflow_addons as tfa 
import matplotlib.pyplot as plt 
import cv2 
import os 
import shutil 
import scipy.io


Chuẩn bị dữ liệu 

In [ ]:
# Path to images and annotations
path_images = "/101_ObjectCategories/airplanes/"
path_annot = "/Annotations/Airplanes_Side_2/"

path_to_downloaded_file = keras.utils.get_file(
    fname="caltech_101_zipped",
    origin="https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip",
    extract=True,
    archive_format="zip",  # downloaded file format
    cache_dir="/",  # cache and extract in current directory
)

# Extracting tar files found inside main zip file
shutil.unpack_archive("/datasets/caltech-101/101_ObjectCategories.tar.gz", "/")
shutil.unpack_archive("/datasets/caltech-101/Annotations.tar", "/")

# danh sách đường dẫn đến hình ảnh và chú thích (annotations)
image_paths = [
    f for f in os.listdir(path_images) if os.path.isfile(os.path.join(path_images, f))
]
annot_paths = [
    f for f in os.listdir(path_annot) if os.path.isfile(os.path.join(path_annot, f))
]

image_paths.sort()
annot_paths.sort()

# resize image 
image_size = 224
images , targets = [] , []
# lọc qua chú thích và hình ảnh , xử lí chúng và lưu trữ trong danh sách 
for i in range(0 ,  len(annot_paths)):
    # acces bounding box coordinates (  truy cập tọa độ hộp giới hạn )
    annot = scipy.io.loadmat(path_annot + annot_paths[i])['box_coord'][0]

    top_left_x , top_left_y = annot[2] ,annot[0]
    bottom_right_x , bottom_right_y = annot[3] , annot[1]

    image = keras.utils.load_img(
        path_images +  image_paths[i]
    )
    (w, h) = image.size[:2]
    # thay đổi kích thước hình ảnh huấn luyện 
    if i < int(len(annot_paths) *0.8):
        # thay đổi kích thước hình ảnh nếu nó cho dữ liệu huấn luyệnn
        image = image.resize(image_size , image_size)
    # chuyển đổi hinh ảnh thành ma trận và append nó vào danh sách 
    images.append(keras.utils.img_to_array(image))

    # áp dụng tỷ lệ tương đối cho hội giới hạn theo hình ảnh cung cấp và áp dụng danh sách 
    targets.append(
        (
            float(top_left_x) / w,
            float(top_left_y) / h,
            float(bottom_right_x) / w,
            float(bottom_right_y) / h,
        )
    )

# chuyển đổi danh sách thành mảng numpy , cắt dữ liệu train và test 
(x_train) , (y_train) = (
    np.asarray(images[: int(len(images)* 0.8)]),
    np.asarray(targets[: int(len(targets) * 0.8)]),
)
(x_test) ,(y_test) = (
    np.asarray(images [int(len(images) *0.8):]),
    np.asarray(targets[int(len(targets) * 0.8):]),
)

Triển khai mlp đa lớp 

In [ ]:
def mlp(x , hidden_units , dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units , activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)

    return x 

Triển khai lớp tạo bản vá 

In [ ]:
patch_size = 32 # kích thươvcs của bản vá lỗi được trích xuất từ ảnh đầu vào 
input_shape = (image_size, image_size, 3)  # input image shape
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 100
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
# Size of the transformer layers
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 4
mlp_head_units = [2048, 1024, 512, 64, 32]  # Size of the dense layers


In [ ]:
class Patches (layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size 

    # ghi đè chức năng để tránh lỗi khi lưu trữ mô hình 
    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {
                "input_shape" : input_shape, 
                "patch_size": patch_size , 
                "num_patches" : num_patches,
                "projection_dim" : projection_dim, 
                "num_heads" : num_heads, 
                "transformer_units" : transformer_units, 
                "transformer_layer" : transformer_layers, 
                "mlp_head_units" : mlp_head_units,
            }
        )
        return config
    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images =images , 
            sizes = [1 , self.patch_size , self.patch_size, 1], 
            srides = [1 , self.patch_size , self.patch_size , 1],
            rates = [ 1 , 1 , 1 , 1],
            padding ='VALID',
        )
        patch_dim = patches.shape[-1] # lấy ra số chiều của ảnh cắt
        return  tf.reshape(patches , [batch_size , -1 , patch_dim])
    

Hiển thị các bản vá

In [ ]:
# hiển thị ảnh gốc shape 4 * 4 xử dụng astype uint8 để lấy các gam màu nguyên bản 
plt.figure(figsize=(4,4))
plt.imshow(x_train[0].astype('uint8'))
plt.axis('off') # bỏ đi trục lưới 

# thực hiện cắt ảnh thành các bản vá và hiển thị các bản vá 
# Chuyển đổi anhr thành 1 tensor có số chiều tương tự 
patches = Patches(patch_size=patch_size)(tf.convert_to_tensor([x_train[0]]))
print(f'image size :{image_size}  x {image_size}')
print(f"patch size : {patch_size} x {patch_size}")
print(f"{patches.shape[1]} patches per image \n{patches.shape[-1]} elements per patch")

# hiển thị các bản vá 
n = int(np.sqrt(patches.shape[-1]))
plt.figure(figsize=(4,4))
for i , patch in enumerate(patches[0]):
    ax = plt.subplot(n , n , i + 1)
    patch_img = tf.reshape(patch , (patch_size , patch_size , 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis('off')

Triển khai lớp mã hóa bản vá 

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches , projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches , output_dim=projection_dim
        )
    # ghi đè hàm để tránh lỗi khi lưu trữ mô hình 
    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {
                "input_shape " :input_shape , 
                "patch_size" : patch_size , 
                "num_patches ": num_patches, 
                "projection_dim" : projection_dim, 
                "num_heads" : num_heads , 
                "transformer_units" : transformer_units, 
                "transformer_layers" : transformer_layers, 
                "mlp_head_units" : mlp_head_units,
            }
        )
        return config 
    def call(self, patch):
        positions = tf.range(start =0 , limit=self.num_patches ,delta=1 )
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded 

Xây dựng mô hình 

In [ ]:
def  create_vit_object_detector(
        input_shape , patch_size , 
        num_patches , projection_dim , 
        num_heads , transformer_units , 
        transformer_layers , 
        mlp_head_units, 
):
    inputs = layers.Input(shape=input_shape)
    # khởi tạo bản vá 
    patches = Patches(patch_size)(inputs)
    # nhúng bản vá 
    encoded_patches = PatchEncoder(num_patches ,projection_dim)(patches)
    # khởi tạo da lớp của khối transformer
    for _ in range (transformer_layers):
        # all layers normalization 
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # create a multi attention_layers
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads , key_dim=projection_dim , dropout=0.1
        )(x1 , x1)
        # skip conection 1 
        x2 = layers.Add()([attention_output , encoded_patches])

        # add layers normalization 2
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # add mlp layers 
        x3 = mlp(x2 ,hidden_units=transformer_units , dropout_rate=0.1)
        # skip connection 2 
        encoded_patches = layers.Add()[x3 , x2]
    # create a tensor [batch_size , projection_dim]
    representations = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representations = layers.Flatten()(representations)
    representations = layers.Dropout(0.3)(representations)

    # add mlp (lớp fully connedted)
    features = mlp(representations , hidden_units=mlp_head_units , dropout_rate=0.3)
    # create bounding_box 
    bounding_box = layers.Dense(4)(
        features 
    ) # 4 tế bào thần kinh cuối cùng là hộp giới hạn 
    # return keras model 
    return keras.Model(inputs=inputs , outputs=bounding_box)


In [ ]:
def run_experiment(model, learning_rate, weight_decay, batch_size, num_epochs):

    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    # Compile model.
    model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError())

    checkpoint_filepath = 'logs/'
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath, 
        monitor ='val_loss',
        save_best_only=True, 
        save_weights_only=True,
    )
    
    history = model.fit(
        x = x_train, 
        y=y_train, 
        batch_size = batch_size ,
        epochs = num_epochs, 
        validation_split = 0.1 ,
        callbacks =[
            checkpoint_callback, 
            keras.callbacks.EarlyStopping(monitor='val_loss' , patience=10)
        ],
    )
    return history 


history = []
num_patches = (image_size // patch_size) ** 2

vit_object_detector = create_vit_object_detector(
    input_shape,
    patch_size,
    num_patches,
    projection_dim,
    num_heads,
    transformer_units,
    transformer_layers,
    mlp_head_units,
)

# Train model
history = run_experiment(
    vit_object_detector, learning_rate, weight_decay, batch_size, num_epochs
)

Đánh giá mô hình

In [ ]:
import matplotlib.patches as patches

# Saves the model in current path
vit_object_detector.save("vit_object_detector.h5", save_format="h5")

def bounding_box_intersection_over_union(box_predicted , box_truth):
    # lấy tọa độ gia điểm của cá hộp giới hạn 
    top_x_intersect = max(box_predicted[0] , box_truth[0])
    top_y_intersect = max(box_predicted[1] , box_truth[1])
    bottom_x_intersect = min(box_predicted[2] , box_truth[2])
    bottom_y_intersect = min(box_predicted[3] , box_truth[3])
    # tính toán diện tích giao nhau 
    intersection_area = max(0 , bottom_x_intersect - top_x_intersect + 1) * max(
        0 , bottom_y_intersect - top_y_intersect + 1
    )
    # tính diện tích của dự đoán và sự thật cơ bản 
    box_predicted_area = (box_predicted[2] - box_predicted[0] +1) * (
        box_predicted[3] - box_predicted[1] + 1 
    )
    # box truth 
    box_truth_area = (box_truth[2] - box_truth[0] + 1) * (box_truth[3] - box_truth[1] + 1)
    # tính toán giao điểm liên hiệp băngd cách lấy diện tích giao nhau 
    # chia cho tổng diện tích dự đoán và diện tích thật diện tích bị trừ bởi diện tích giao nhau 
    # iou = A ∩ B / A u B
    # diện tích hợp của hộp dự đoán và hộp thực = tổng dự đoán + thực tế trừ đi phần giao 
    return intersection_area / float (
        box_truth_area + box_predicted_area - intersection_area
    )

i , mean_iou = 0, 0
# compare result for 10 images in the tes set 
# so sánh kết quả cho 10 ảnh trong tệp thử nghiệm 

for input_image in x_test[:10]:
    fig ,(ax1 , ax2) = plt.subplots( 1, 2 , figsize=(15 ,15))
    im = input_image 

    # hiển thị hình ảnh 
    ax1.imshow(im.astype('uint8'))
    ax2.imshow(im.astype('uint8'))
    
    # thay đổi kích thước hình ảnh đầu vào 
    input_image = cv2.resize(
        input_image , (image_size , image_size) , interpolation=cv2.INTER_AREA
    )
    # thêm 1 chiều rỗng vào ảnh batch_size  để biến hình ảnh thành [ 1 , h , w] 
    input_image = np.expand_dims(input_image, axis=0)
    # dự đoán hình ảnh và lấy phần tử đầu tiên của danh sách 
    preds = vit_object_detector.predict(input_image)[0] 
    # lấy ra tọa độ h và w của hình ảnh 
    (h , w) = (im).shape[0:2] # shape [0:2] lấy 2 phần tử đầu tiên của tuple tương ứng với h và w
    # biến đổi  x , y thành tọa độ số nguyên 
    top_left_x , top_left_y = int(preds[0] * w) , int(preds[1] * h)
    
    bottom_right_x , bottom_right_y = int(preds[2] *w) , int(preds[3] *h)

    # lấy ra hộp giới hạn dự đoán 
    box_predicted = [top_left_x , top_left_y , bottom_right_x ,bottom_right_y]

    # khởi tạo hộp giới hạn
    rect = patches.Rectangle(
        (top_left_x , top_left_y),
        bottom_right_x - top_left_x ,
        bottom_right_y - top_left_y, 
        facecolor ='none',
        edgecolor ='green', 
        linewwidth=1,
    )
    # thêm hộp giới hạn vào hình ảnh
    ax1.add_patch(rect)
    ax1.set_xlabel(
        "Predicted: "
        + str(top_left_x)
        + ", "
        + str(top_left_y)
        + ", "
        + str(bottom_right_x)
        + ", "
        + str(bottom_right_y)
    )

    top_left_x , top_left_y = int(y_test[i][0] * w) , int(y_test[i][1] * h)
    bottom_right_x, bottom_right_y = int(y_test[i][2] * w), int(y_test[i][3] * h)

    box_truth = top_left_x, top_left_y, bottom_right_x, bottom_right_y

    mean_iou += bounding_box_intersection_over_union(box_predicted , box_truth)

    # create the bounding box 
    rect = patches.Rectangle(
        (top_left_x, top_left_y),
        bottom_right_x - top_left_x, 
        bottom_right_y - top_left_y, 
        facecolor ='none',
        adgecolor='red',
        linewidth=1,
    )
    # add the bounding box
    ax2.add_patch(rect)
    ax2.set_xlabel(
        "Target: "
        + str(top_left_x)
        + ", "
        + str(top_left_y)
        + ", "
        + str(bottom_right_x)
        + ", "
        + str(bottom_right_y)
        + "\n"
        + "IoU"
        + str(bounding_box_intersection_over_union(box_predicted, box_truth))
    )
    i = i + 1

print("mean_iou :" + str(mean_iou / len(x_test[:10])))
plt.show()